In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Housing.csv')

# Display the first 5 rows of the DataFrame
#print(df.head())

In [6]:
from IPython.display import display
display(df)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [7]:
# List of variables to map

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
df[varlist] = df[varlist].apply(binary_map)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [47]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'price']

df[num_vars] = scaler.fit_transform(df[num_vars])
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.000000,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,furnished
1,0.909091,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,furnished
2,0.909091,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,semi-furnished
3,0.906061,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,furnished
4,0.836364,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,furnished


In [24]:
input = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

#print(df.columns[1])
X = df.values[:, 1]  # get input values from the first column -- X is a list here which is a 1 dimentional array
y = df.values[:, 0]  # get output values from the second column --  Y is a list here which is a 2 dimentional array
X_3 = df.values[:, 1:5]
X_1 = df.values[:, 10].reshape(545, 1)
#print(X_1)
X = np.hstack((X_3, X_1))
X = torch.from_numpy(X.astype(np.float64))
y = torch.from_numpy(y.astype(np.float64))

split = int(0.8*545)
shuffled = torch.randperm(545)
train = shuffled[:split]
val = shuffled[split:]

X_train = X[train]
y_train = y[train]
X_val = X[val]
y_val = y[val]

In [106]:
def model(x, w1, w2, w3, w4, w5, b):
    return w1*x[:,0] + w2*x[:,1] + w3*x[:,2] + w4*x[:,3] + w5*x[:,4] + b

def loss_fn(p, c):
    squared_diffs = (p - c)**2
    return squared_diffs.mean()

In [107]:
import torch.optim as optim

params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

t_p = model(X_train, *params)
loss = loss_fn(t_p, y_train)
loss.backward()

optimizer.step()

params

tensor([ 0.9917,  0.9864,  0.9960,  0.9888,  0.9695, -0.0322],
       requires_grad=True)

In [104]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(X_train, *params) # <1>
        train_loss = loss_fn(train_t_p, y_train)

        val_t_p = model(X_val, *params) # <1>
        val_loss = loss_fn(val_t_p, y_val)

        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"f" Validation loss {val_loss.item():.4f}")

    return params

SGD

In [108]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-3
optimizer = optim.SGD([params], lr=learning_rate)
print('LR:', learning_rate)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = X_train, # <1>
    val_t_u = X_val, # <1>
    train_t_c = y_train,
    val_t_c = y_val)


LR: 0.001
Epoch 500, Training loss 0.1442, Validation loss 0.1339
Epoch 1000, Training loss 0.0919, Validation loss 0.0956
Epoch 1500, Training loss 0.0813, Validation loss 0.0863
Epoch 2000, Training loss 0.0726, Validation loss 0.0777
Epoch 2500, Training loss 0.0652, Validation loss 0.0702
Epoch 3000, Training loss 0.0587, Validation loss 0.0637
Epoch 3500, Training loss 0.0532, Validation loss 0.0581
Epoch 4000, Training loss 0.0483, Validation loss 0.0533
Epoch 4500, Training loss 0.0441, Validation loss 0.0491
Epoch 5000, Training loss 0.0405, Validation loss 0.0454


tensor([ 0.7216,  0.5624,  0.6914,  0.3214,  0.2083, -0.4895],
       requires_grad=True)

ADAM

In [109]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = optim.Adam([params], lr=learning_rate) # <1>
print('LR:', learning_rate)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = X_train, # <1>
    val_t_u = X_val, # <1>
    train_t_c = y_train,
    val_t_c = y_val)

LR: 0.0001
Epoch 500, Training loss 2.4484, Validation loss 2.2461
Epoch 1000, Training loss 2.0375, Validation loss 1.8624
Epoch 1500, Training loss 1.6781, Validation loss 1.5277
Epoch 2000, Training loss 1.3651, Validation loss 1.2370
Epoch 2500, Training loss 1.0943, Validation loss 0.9865
Epoch 3000, Training loss 0.8623, Validation loss 0.7726
Epoch 3500, Training loss 0.6659, Validation loss 0.5927
Epoch 4000, Training loss 0.5026, Validation loss 0.4440
Epoch 4500, Training loss 0.3698, Validation loss 0.3242
Epoch 5000, Training loss 0.2649, Validation loss 0.2306


tensor([ 0.5843,  0.5843,  0.5787,  0.5774,  0.5827, -0.4124],
       requires_grad=True)

Problem 3

In [64]:
input = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
       'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']

#print(df.columns[1])
X = df.values[:, 1:12]  # get input values from the first column -- X is a list here which is a 1 dimentional array
y = df.values[:, 0]  # get output values from the second column --  Y is a list here which is a 2 dimentional array

X = torch.from_numpy(X.astype(np.float64))
y = torch.from_numpy(y.astype(np.float64))

split = int(0.8*545)
shuffled = torch.randperm(545)
train = shuffled[:split]
val = shuffled[split:]

X_train = X[train]
y_train = y[train]
X_val = X[val]
y_val = y[val]

In [65]:
weights = np.ones(12)
np.append(weights, 0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.])

In [76]:
def model(x, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, b):
    return w1*x[:,0] + w2*x[:,1] + w3*x[:,2] + w4*x[:,3] + w5*x[:,4] + w6*x[:,5] + w7*x[:,6] + w8*x[:,7] + w9*x[:,8] + w10*x[:,9] + w11*x[:,10] + b

def loss_fn(p, c):
    squared_diffs = (p - c)**2
    return squared_diffs.mean()

In [77]:
import torch.optim as optim

weights = np.ones(11)
weights = np.append(weights, 0)
params = torch.tensor(weights, requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params], lr=learning_rate)

t_p = model(X_train, *params)
loss = loss_fn(t_p, y_train)
loss.backward()

optimizer.step()

params

tensor([ 0.9841,  0.9750,  0.9926,  0.9807,  0.9446,  0.9837,  0.9731,  0.9963,
         0.9754,  0.9832,  0.9786, -0.0593], dtype=torch.float64,
       requires_grad=True)

In [101]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(X_train, *params) # <1>
        train_loss = loss_fn(train_t_p, y_train)

        val_t_p = model(X_val, *params) # <1>
        val_loss = loss_fn(val_t_p, y_val)

        optimizer.zero_grad()
        train_loss.backward() # <2>
        optimizer.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"f" Validation loss {val_loss.item():.4f}")

    return params

SGD

In [102]:
weights = np.ones(11)
weights = np.append(weights, 0)
params = torch.tensor(weights, requires_grad=True)
learning_rate = 1e-4
optimizer = optim.SGD([params], lr=learning_rate)
print('LR:', learning_rate)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = X_train, # <1>
    val_t_u = X_val, # <1>
    train_t_c = y_train,
    val_t_c = y_val)

LR: 0.0001
Epoch 500, Training loss 6.7046, Validation loss 6.5288
Epoch 1000, Training loss 4.2640, Validation loss 4.1322
Epoch 1500, Training loss 2.7835, Validation loss 2.6813
Epoch 2000, Training loss 1.8827, Validation loss 1.8009
Epoch 2500, Training loss 1.3321, Validation loss 1.2647
Epoch 3000, Training loss 0.9932, Validation loss 0.9362
Epoch 3500, Training loss 0.7823, Validation loss 0.7330
Epoch 4000, Training loss 0.6489, Validation loss 0.6054
Epoch 4500, Training loss 0.5626, Validation loss 0.5235
Epoch 5000, Training loss 0.5050, Validation loss 0.4693


tensor([ 0.7018,  0.5455,  0.8385,  0.6221,  0.0340,  0.6219,  0.4412,  0.9116,
         0.4776,  0.6583,  0.5367, -1.0066], dtype=torch.float64,
       requires_grad=True)

In [103]:
weights = np.ones(11)
weights = np.append(weights, 0)
params = torch.tensor(weights, requires_grad=True)
learning_rate = 1e-3
optimizer = optim.Adam([params], lr=learning_rate) # <1>
print('LR:', learning_rate)
training_loop(
    n_epochs = 5000,
    optimizer = optimizer,
    params = params,
    train_t_u = X_train, # <1>
    val_t_u = X_val, # <1>
    train_t_c = y_train,
    val_t_c = y_val)

LR: 0.001
Epoch 500, Training loss 2.0799, Validation loss 2.0221
Epoch 1000, Training loss 0.2599, Validation loss 0.2570
Epoch 1500, Training loss 0.0808, Validation loss 0.0891
Epoch 2000, Training loss 0.0569, Validation loss 0.0656
Epoch 2500, Training loss 0.0420, Validation loss 0.0495
Epoch 3000, Training loss 0.0320, Validation loss 0.0382
Epoch 3500, Training loss 0.0256, Validation loss 0.0307
Epoch 4000, Training loss 0.0212, Validation loss 0.0256
Epoch 4500, Training loss 0.0179, Validation loss 0.0216
Epoch 5000, Training loss 0.0152, Validation loss 0.0185


tensor([ 0.3213,  0.3830,  0.1789,  0.1301,  0.2449,  0.0120,  0.0670,  0.0843,
         0.0682,  0.0689,  0.0240, -0.3193], dtype=torch.float64,
       requires_grad=True)